In [2]:
import numpy as np
import pandas as pd
import datetime
import copy
import time
import os
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import operator

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
from datetime import timedelta

tqdm.pandas()

In [3]:
# Edit to point to your MIMIC directory.
dataDirStr = '/Users/gmessier/data/mimic-1.4/'

In [5]:
callout_df = pd.read_csv(dataDirStr + "CALLOUT.csv")
callout_df.columns = callout_df.columns.str.lower()
callout_df

,row_id,subject_id,hadm_id,submit_wardid,submit_careunit,curr_wardid,curr_careunit,callout_wardid,callout_service,request_tele,...,callout_status,callout_outcome,discharge_wardid,acknowledge_status,createtime,updatetime,acknowledgetime,outcometime,firstreservationtime,currentreservationtime
0,402,854,175684,52.0,NaN,29.0,MICU,1,MED,0,...,Inactive,Discharged,29.0,Acknowledged,2146-10-05 13:16:55,2146-10-05 13:16:55,2146-10-05 13:24:00,2146-10-05 18:55:22,2146-10-05 15:27:44,NaN
1,403,864,138624,15.0,NaN,55.0,CSRU,55,CSURG,0,...,Inactive,Discharged,55.0,Acknowledged,2114-11-28 08:31:39,2114-11-28 09:42:08,2114-11-28 09:43:08,2114-11-28 12:10:02,NaN,NaN
2,404,864,138624,12.0,NaN,55.0,CSRU,55,CSURG,1,...,Inactive,Discharged,55.0,Acknowledged,2114-11-30 10:24:25,2114-12-01 09:06:18,2114-12-01 12:26:05,2114-12-01 21:55:05,NaN,NaN
3,405,867,184298,7.0,NaN,17.0,CCU,17,CCU,1,...,Inactive,Discharged,17.0,Acknowledged,2136-12-29 08:45:42,2136-12-29 10:17:16,2136-12-29 10:33:51,2136-12-29 18:10:02,NaN,NaN
4,157,306,167129,57.0,NaN,3.0,SICU,44,NSURG,1,...,Inactive,Discharged,3.0,Acknowledged,2199-09-18 11:47:47,2199-09-18 11:47:47,2199-09-18 11:58:33,2199-09-18 15:10:02,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34494,32674,94046,199742,23.0,NaN,54.0,MICU,1,MED,1,...,Inactive,Discharged,54.0,Acknowledged,2112-11-01 11:13:04,2112-11-01 11:13:04,2112-11-01 11:16:14,2112-11-01 19:42:07,2112-11-01 16:12:28,NaN
34495,32675,94049,135809,23.0,NaN,45.0,MICU,1,MED,0,...,Inactive,Discharged,45.0,Acknowledged,2112-10-09 14:12:09,2112-10-09 14:12:09,2112-10-09 14:23:52,2112-10-09 17:40:05,NaN,NaN
34496,32676,94050,173013,23.0,NaN,45.0,MICU,1,MED,0,...,Inactive,Discharged,45.0,Acknowledged,2165-10-07 10:18:42,2165-10-07 10:18:42,2165-10-07 10:20:31,2165-10-07 19:10:11,2165-10-07 17:10:10,NaN
34497,32677,94056,172374,50.0,NaN,45.0,MICU,1,MED,0,...,Inactive,Discharged,45.0,Acknowledged,2199-10-30 15:35:30,2199-10-30 15:35:30,2199-10-30 16:16:09,2199-10-30 22:10:04,NaN,NaN


`CALLOUT.csv`  provides information about a patient's ICU discharge. When a patient is deemed ready to leave the ICU, they are "called out". A discharge does not necessarily mean that the patient will be going home, callouts also include moving between other distinct wards in the hospital.

In [6]:
print(f"There are {callout_df.subject_id.nunique()} unique patients who have been called out")

There are 22871 unique patients who have been called out


`submit_wardid` and identify the ward from which the request was submitted and `submit_careunit` identifies whether the `submit_wardid` corresponds to an ICU cost center.

`submit_wardid` is numerical categorical data, representing 1 of 9 wards in the hospital.
`submit_careunit` is categorical data, with possible 6 values.

In [7]:
c = callout_df.submit_wardid.value_counts().nlargest(5)
p = callout_df.submit_wardid.value_counts(normalize=True).nlargest(5).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
52.0,5598,16.23
14.0,4305,12.48
23.0,4252,12.33
7.0,4066,11.79
50.0,4050,11.74


In [8]:
c = callout_df.submit_careunit.value_counts(dropna=False)[:5]
p = callout_df.submit_careunit.value_counts(dropna=False, normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
NaN,29377,85.15
MICU,2263,6.56
SICU,859,2.49
CSRU,770,2.23
CCU,660,1.91


`curr_wardid` identifies the ward in which the patient resides when "called out" (prior to discharge/transfer). `curr_careunit` indicates which ICU cost center the `curr_wardid` corresponds to. In this case, there should not be a "None" since all patients being discharged from the ICU should reside in an ICU cost center.

`curr_wardid` is similar to `submit_wardid`, numerical categorical data with a possible 9 values.
`curr_careunit` is similar to `submit_careunit`, categorical data with a possible 6 values.

In [9]:
c = callout_df.curr_wardid.value_counts().nlargest(5)
p = callout_df.curr_wardid.value_counts(normalize=True).mul(100).round(2).nlargest(5)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
2.0,3514,10.19
55.0,3512,10.18
45.0,3512,10.18
31.0,2921,8.47
40.0,2378,6.89


In [10]:
c = callout_df.curr_careunit.value_counts(dropna=False)[:5]
p = c = callout_df.curr_careunit.value_counts(dropna=False, normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
MICU,40.62,40.62
SICU,19.27,19.27
CSRU,16.04,16.04
TSICU,12.28,12.28
CCU,11.79,11.79


`callout_wardid` is numerical categorical data which identifies the ward to which the patient should be discharged. A value of 0 represents "Home" and a value of 1 respresents "First available ward". The remaining numbers correspond to distinct wards in the hospital. `callout_service` is the service under which the patient should be discharged.

In [11]:
c = callout_df.callout_wardid.value_counts()[:5]
p = callout_df.callout_wardid.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
1,16280,47.19
2,3801,11.02
31,2683,7.78
17,2361,6.84
40,2024,5.87


In [12]:
c = callout_df.callout_service.value_counts()[:5]
p = callout_df.callout_service.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
MED,13791,39.98
CSURG,4782,13.86
CCU,2993,8.68
SURG,2902,8.41
NSURG,2234,6.48


`request_tele`, `request_resp`, `request_cdiff`, `reuqest_mrsa`, `request_vre` are all binary indicators which request for certain percautions for the patients in the subsequente ward where they will reside. 

`callout_status` indicates whether the callout is still active, if the callout is completed it will be marked as inactive. `callout_outcome` indicates whether the patient was actually discharged or not.

In [13]:
c = callout_df.callout_status.value_counts()
p = callout_df.callout_status.value_counts(normalize=True).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
Inactive,34499,100.0


In [14]:
c = callout_df.callout_outcome.value_counts()
p = callout_df.callout_outcome.value_counts(normalize=True).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
Discharged,29965,86.86
Cancelled,4534,13.14


`discharge_wardid` indicates the ward to which the patient was actually discharged. `discharge_wardid` = 0 indicates home and other values correspond to distinct wards in the hospital

In [15]:
c = callout_df.discharge_wardid.value_counts().nlargest(5)
p = callout_df.discharge_wardid.value_counts(normalize=True).mul(100).round(2).nlargest(5)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
2.0,3513,11.72
55.0,3509,11.71
45.0,3508,11.71
31.0,2918,9.74
40.0,2375,7.93


`acknowledge_status` indicates the response to the callout event: ‘Acknowledged’, ‘Revised’, ‘Unacknowledged’ or ‘Reactivated’.

In [16]:
c = callout_df.acknowledge_status.value_counts()
p = callout_df.acknowledge_status.value_counts(normalize=True).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
Acknowledged,32707,94.81
Unacknowledged,1083,3.14
Revised,657,1.90
Reactivated,52,0.15


`createtime` provides the time and date that the call out was initiated. 

`updatetime` provides the last time at which the call out event was updated. 

`acknowledgetime` is the time at which the call out was first acknowledged. 

`outcometime` is the time at which the `callout_outcome` occurred.

`firstreservationtime` and `currentreservationtime` provide information regarding ward reservations.